In [1]:
import T4sigWGAN as T4
from torch.utils.data import DataLoader, random_split

In [9]:
train_dataset = T4.StockTimeSeriesDataset()

train_size = int(0.9 * len(train_dataset))  # 90% for training
val_size = len(train_dataset) - train_size  # 10% for validation
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoader 설정
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)

In [2]:
Encoder = T4.LogSigRNNDecoder(**T4.encoder_config).to(T4.args.device)
Encoder

LogSigRNNDecoder(
  (rnn): Sequential(
    (0): FeedForwardNN(
      (network): Sequential(
        (0): Linear(in_features=466, out_features=256, bias=True)
        (1): PReLU(num_parameters=1)
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): PReLU(num_parameters=1)
        (4): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): Tanh()
  )
  (linear): Linear(in_features=256, out_features=256, bias=False)
  (initial_nn): Sequential(
    (0): ResidualNN(
      (network): Sequential(
        (0): ResidualBlock(
          (activation): ReLU()
          (linear): Linear(in_features=10, out_features=256, bias=True)
        )
        (1): ResidualBlock(
          (activation): ReLU()
          (linear): Linear(in_features=256, out_features=256, bias=True)
        )
        (2): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): Tanh()
  )
)

In [3]:
Decoder = T4.TimesFormerDecoder(**T4.decoder_config).to(T4.args.device)
Decoder

TimesFormerDecoder(
  (timesformer_blocks): ModuleList(
    (0): TimesFormerLayer(
      (conv): Conv1d(256, 256, kernel_size=(4,), stride=(1,), padding=same)
      (relu): ReLU()
      (norm_tcn): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (mha1): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ff_conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (ff_relu): ReLU()
      (ff_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (cross1): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (cross2): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elem

In [4]:
Supervisor = T4.ModernTCN(T4.supervisor_config).to(T4.args.device)
Supervisor

ModernTCN(
  (revin): RevIN()
  (decomp): series_decomp(
    (moving_avg): moving_avg(
      (avg): AvgPool1d(kernel_size=(25,), stride=(1,), padding=(0,))
    )
  )
  (conv_layers): ModuleList(
    (0): ReparamLargeKernelConv(
      (lkb_origin): Sequential(
        (conv): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (small_conv): Sequential(
        (conv): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ReparamLargeKernelConv(
      (lkb_origin): Sequential(
        (conv): Conv1d(256, 128, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (small_conv): Sequential(
        (conv): Conv1d(256

In [5]:
Generator = T4.LogSigRNNGenerator(**T4.logsig_config).to(T4.args.device)
Generator

LogSigRNNGenerator(
  (rnn): Sequential(
    (0): FeedForwardNN(
      (network): Sequential(
        (0): Linear(in_features=466, out_features=256, bias=True)
        (1): PReLU(num_parameters=1)
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): PReLU(num_parameters=1)
        (4): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): Tanh()
  )
  (linear): Linear(in_features=256, out_features=256, bias=False)
  (initial_nn): Sequential(
    (0): ResidualNN(
      (network): Sequential(
        (0): ResidualBlock(
          (activation): ReLU()
          (linear): Linear(in_features=10, out_features=256, bias=True)
        )
        (1): ResidualBlock(
          (activation): ReLU()
          (linear): Linear(in_features=256, out_features=256, bias=True)
        )
        (2): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (1): Tanh()
  )
)

In [6]:
Discriminator = T4.tailGANDiscriminator(T4.discriminator_config).to(T4.args.device)
Discriminator

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [7]:
model = T4.T4sigWGAN(Encoder, Decoder, Generator, Supervisor, Discriminator).to(T4.args.device)
trainer = T4.FinetuneTrainer(T4.args, model, train_dataloader, val_dataloader)

In [ ]:
stage = "Pretrain_1"

# model training
for epoch in range(T4.args.epochs):
    trainer.train(epoch, stage)
    val_loss = trainer.valid(epoch, stage)

# train log image save
trainer.evaluate()

In [ ]:
stage = "Pretrain_2"

# model training
for epoch in range(T4.args.epochs):
    trainer.train(epoch, stage)
    val_loss = trainer.valid(epoch, stage)

# train log image save
trainer.evaluate()

In [ ]:
stage = "Finetune"

# model training
for epoch in range(T4.args.epochs):
    trainer.train(epoch, stage)
    val_loss = trainer.valid(epoch, stage)

# train log image save
trainer.evaluate()

In [ ]:
# metric_iteration = 5
#
# discriminative_score = list()
# for _ in range(metric_iteration):
#     temp_disc = T4.discriminative_score_metrics(ori_data, generated_data)
#     discriminative_score.append(temp_disc)
#
# print('Discriminative score: ' + str(np.round(np.mean(discriminative_score), 4)))
#
#
# predictive_score = list()
# for tt in range(metric_iteration):
#     temp_pred = T4.predictive_score_metrics(ori_data, generated_data)
#     predictive_score.append(temp_pred)
#
# print('Predictive score: ' + str(np.round(np.mean(predictive_score), 4)))
#
#
# T4.visualization(ori_data, generated_data, 'pca')
# T4.visualization(ori_data, generated_data, 'tsne')